In [1]:
import json
import torch 
import numpy as np
import einops   

In [5]:
with open("/home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers/relation-analysis/outputs/stage_a/vg_stage_a_full.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

In [7]:
len(data)

1342859

In [17]:
#unique predicates
from tqdm import tqdm
predicates = set()
for item in tqdm(data):
    # print(item)
    predicates.add(item["triple"]["predicate"])
print(len(predicates))

  0%|          | 0/1342859 [00:00<?, ?it/s]

100%|██████████| 1342859/1342859 [00:00<00:00, 1724304.80it/s]

24


In [16]:
data[0]

{'triple': {'subject': 'shade',
  'predicate': 'on',
  'object': 'sidewalk',
  'relation_type': 'geometric',
  'source_image_id': 1,
  'source_relationship_id': 15927},
 'prompt': 'a photo of shade on sidewalk on a plain background',
 'concepts': ['shade', 'on', 'sidewalk'],
 'template_id': 'simple_v1'}

In [18]:
# Get all predicates and their counts
predicate_counts = {}
for item in tqdm(data, desc="Counting predicates"):
    pred = item["triple"]["predicate"]
    predicate_counts[pred] = predicate_counts.get(pred, 0) + 1

# Sort by count
sorted_predicates = sorted(predicate_counts.items(), key=lambda x: x[1], reverse=True)
print("\nPredicate distribution:")
for pred, count in sorted_predicates:
    print(f"{pred:20s}: {count:6d} samples")

Counting predicates: 100%|██████████| 1342859/1342859 [00:01<00:00, 1193031.23it/s]


Predicate distribution:
on                  : 668005 samples
in                  : 231977 samples
wearing             : 140590 samples
around/near         :  67126 samples
above               :  54393 samples
behind              :  40452 samples
holding             :  37909 samples
below               :  23187 samples
sitting on          :  13893 samples
hanging from        :  13810 samples
in front of         :  13367 samples
standing on         :   7647 samples
riding              :   7580 samples
looking at          :   6541 samples
carrying            :   4799 samples
eating              :   4507 samples
using               :   1759 samples
pulling             :   1178 samples
touching            :   1133 samples
playing with        :    914 samples
drinking            :    647 samples
left of             :    614 samples
pushing             :    430 samples
right of            :    401 samples


# Dataset Generation Scripts Created

I've created a complete multi-GPU pipeline for generating saliency map datasets. Here's what's available:

## 📁 Files Created

1. **`generate_saliency_dataset.py`** - Main generation script
   - Uses 4 GPUs in parallel
   - Generates 2000 samples per relationship class
   - Creates 3 separate datasets (early/middle/late layers)

2. **`dataset_loader.py`** - PyTorch Dataset class and utilities
   - Load datasets for training
   - Verify dataset integrity
   - View statistics and samples

3. **`test_setup.py`** - Pre-flight test script
   - Verifies everything works before full generation
   - Tests pipeline, data loading, and save/load

4. **`DATASET_README.md`** - Complete documentation

## 🚀 Quick Start

### Step 1: Test the setup (RECOMMENDED)
```bash
cd /home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers
python test_setup.py
```

### Step 2: Generate all datasets
```bash
python generate_saliency_dataset.py --layer-config all --num-gpus 4
```

This will create:
```
saliency_datasets/
├── early_layers/    # Layers 0-6
├── middle_layers/   # Layers 7-12
└── late_layers/     # Layers 13-18
```

## 📊 Dataset Details

- **24 relationship classes** (on, in, wearing, holding, etc.)
- **2000 samples per class** = 48,000 samples per dataset
- **3 datasets total** = 144,000 samples
- **512x512 images** → 32x32 saliency maps
- **Multi-GPU**: 500 samples per GPU (parallel processing)

## ⏱️ Estimated Time

With 4 GPUs (assuming ~3-5 sec/sample):
- Per class: ~40-60 minutes
- Per dataset: ~16-24 hours
- All 3 datasets: ~48-72 hours

## 💾 Storage

- ~150-250MB per 1000 samples
- Total: ~30-50GB for all 3 datasets

## 🔧 Advanced Usage

### Generate only one dataset
```bash
python generate_saliency_dataset.py --layer-config early_layers
```

### Use fewer GPUs
```bash
python generate_saliency_dataset.py --num-gpus 2
```

### Custom sample count
```bash
python generate_saliency_dataset.py --samples-per-class 1000
```

### Verify generated dataset
```bash
python dataset_loader.py --dataset-path saliency_datasets/early_layers --action verify
```

## 📈 Using in Training

```python
from dataset_loader import SaliencyMapDataset
from torch.utils.data import DataLoader

dataset = SaliencyMapDataset("saliency_datasets/early_layers")
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    saliency = batch["saliency_maps"]  # (B, 3, 32, 32)
    labels = batch["label"]            # (B,)
    # Train your classifier...
```